# Chain-of-Thought (CoT) Loan Application – Tools Needed (OpenAI Version)

This notebook demonstrates a **CoT** approach for a **loan application** scenario, but the agent **only** identifies which tools to invoke—**no** final “Approved” or “Denied” decision.

## How It Works
1. We have **loan criteria** checks for credit score, annual income, and debt-to-income ratio.
2. We call **OpenAI** (with model `gpt-4o`, or change it to `gpt-3.5-turbo`) for a chain-of-thought.
3. The LLM produces a **step-by-step** reasoning plus a line: `The tools to invoke are: ...`.
4. We parse that line, but **don’t** actually call the tools.

You can tweak the applicant data to see how the LLM suggests different tools (e.g., `CreditBureauAPI`, `CoSignerLookup`, etc.).

## 1. Loan Criteria
We define a minimal **domain** logic for **loan eligibility**:
- `MIN_CREDIT_SCORE = 650`
- `MIN_ANNUAL_INCOME = 30000`
- `MAX_DEBT_TO_INCOME = 0.4` (40%)

We have a function `basic_loan_eligibility_check` that returns which checks pass or fail.

In [ ]:
LOAN_POLICY = {
    "MIN_CREDIT_SCORE": 650,       # Minimum acceptable credit score
    "MIN_ANNUAL_INCOME": 30000,   # Minimum annual income in dollars
    "MAX_DEBT_TO_INCOME": 0.4     # 40% maximum debt-to-income ratio
}

def basic_loan_eligibility_check(credit_score, annual_income, monthly_debt, monthly_income):
    if monthly_income > 0:
        dti = monthly_debt / monthly_income
    else:
        dti = 1.0  # fail if monthly_income <= 0

    checks = {
        "credit_score_check": credit_score >= LOAN_POLICY["MIN_CREDIT_SCORE"],
        "income_check": annual_income >= LOAN_POLICY["MIN_ANNUAL_INCOME"],
        "dti_check": dti <= LOAN_POLICY["MAX_DEBT_TO_INCOME"]
    }
    return checks

## 2. OpenAI Adapter
We create a function `call_openai_for_tools` that:
1. Sends a **prompt** to your **`gpt-4o`** model (adjustable if needed).
2. Expects a **chain-of-thought** plus `The tools to invoke are: ...` line in the response.
3. Returns the raw text.

> Make sure you have `OPENAI_API_KEY` set or pass an API key to `openai.api_key`.

We'll parse the final message in the **agent** class next.

In [ ]:
from openai import OpenAI
import os
import re

os.environ['OPENAI_API_KEY']=''

def call_openai_for_tools(prompt, model="gpt-4o", temperature=0.2):
    system_message = "You are a chain-of-thought loan analysis AI. Instead of Approving/Denial, we only pick which tools we need." 
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]

    client = OpenAI()

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=300,
    )

    return response.choices[0].message.content.strip()

## 3. CoT Agent
1. **Runs** the basic checks for credit score, income, DTI.
2. **Prompts** OpenAI for a chain-of-thought plus tool suggestions.
3. **Parses** out the final tools from the text.

We do **not** call or implement the tools—just identifying them.

In [ ]:
class LoanCoTAgent:
    def __init__(self, model="gpt-4o"):
        self.model = model
        self.conversation_log = []

    def process_loan_application(self, credit_score, annual_income, monthly_debt, monthly_income):
        checks = basic_loan_eligibility_check(
            credit_score,
            annual_income,
            monthly_debt,
            monthly_income
        )

        # Build a context string
        context_info = (
            f"Applicant's credit score: {credit_score}\n"
            f"Applicant's annual income: {annual_income}\n"
            f"Applicant's monthly debt: {monthly_debt}\n"
            f"Applicant's monthly income: {monthly_income}\n"
            "Checks:\n"
            f" - credit_score_check: {checks['credit_score_check']}\n"
            f" - income_check: {checks['income_check']}\n"
            f" - dti_check: {checks['dti_check']}\n"
        )

        prompt = (
            "Please reason step-by-step about which tools are needed to process this loan.\n"
            "If any checks fail, we might need more verification or partner tools.\n"
            "Your response should include chain-of-thought and end with:\n"
            "'The tools to invoke are: ...'\n"
            "(No actual invocation needed.)\n\n"
            f"{context_info}\n"
            "Please reason step-by-step"
        )

        response_text = call_openai_for_tools(prompt, model=self.model)
        self.conversation_log.append(response_text)

        # By default, just store the entire response in 'chain_of_thought'.
        chain_of_thought = response_text
        tools = "None"

        if "The tools to invoke are:" in response_text:
            tools_part = response_text.split("The tools to invoke are:")[-1].strip()
            tools = tools_part if tools_part else "None"

        return {
            "chain_of_thought": chain_of_thought,
            "tools": tools
        }

## 4. Main Orchestration
We simulate some applicant data, call our **LoanCoTAgent**, and print out the chain-of-thought plus identified tools.

Change the user’s credit score, income, or debt to see how the recommended tools might vary.

In [ ]:
if __name__ == "__main__":
    # Sample applicant data
    applicant_credit_score = 680
    applicant_annual_income = 28000.0  # below the min threshold
    applicant_monthly_debt = 800.0
    applicant_monthly_income = 3000.0

    agent = LoanCoTAgent(model="gpt-4o")
    result = agent.process_loan_application(
        credit_score=applicant_credit_score,
        annual_income=applicant_annual_income,
        monthly_debt=applicant_monthly_debt,
        monthly_income=applicant_monthly_income
    )

    print("=== CHAIN OF THOUGHT + TOOLS ===\n")
    print(result["chain_of_thought"], "\n")
    print(f"Tools: {result['tools']}")

## Usage
1. **Install** `openai` and set your `OPENAI_API_KEY` environment variable (or pass it in code).
2. **Run** the notebook. The final cell prints the chain-of-thought plus whichever tools the LLM says to invoke.
3. Adjust the applicant data to see changes (e.g., if the credit score is under 650, it might add a "CreditBureauAPI").

## Example Output
You might see something like:
```
=== CHAIN OF THOUGHT + TOOLS ===

Chain of Thought:
1. The user's credit score is okay, but the annual income is below 30k.
Therefore, we might need a cosigner.
The tools to invoke are: CoSignerLookup

Tools: CoSignerLookup
```

## Key Takeaways
1. **One-Pass**: We only decide which tools, no loops or actual tool usage.
2. **CoT**: The LLM shows step-by-step reasoning, then a final line indicating needed tools.
3. **Easily Extended**: Add more conditions or more tools as your logic grows.
4. **Model**: We used `gpt-4o` (per your snippet). If you prefer `gpt-3.5-turbo`, just change `model="gpt-3.5-turbo"`.